# Grid Search for Binary Classification

This notebook goes through applying the `GridSearch` algorithm in Fairlearn to a binary classification problem, where we also have a binary protected attribute. This algorithm comes from the paper ["A Reductions Approach to Fair Classification" (Agarwal et al. 2018)](https://arxiv.org/abs/1803.02453). The grid search is a simplified version of the full algorithm (appearing in section 3.4), which works best for binary classification and a binary protected attribute.

The specific problem we consider a biased credit scoring problem. We assume that we have a collection of individuals characterised by two features - a credit score in the range $[0, 1]$ and a binary protected attribute from ${a_0, a_1}$. We also have a binary 'score' for each individual indicating whether or not they got a loan. This is determined by applying a threshold to their credit score, and to make the dataset unfair, we can set different thresholds for the two groups $a_0$ and $a_1$.

In this simple case, we make the protected attribute and the credit score available to both the estimator and the model. This gives us a straightforward method of assessing fairness - look at the model, and see if zero weight is put on the protected attribute.

In [ ]:
from fairlearn.reductions import GridSearch
from fairlearn.reductions import DemographicParity

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

We use the following function to create the input data. The credit scores for each population are uniformly distributed in the range $[0, 1]$, and we apply separate thresholds to each subpopulation to determine the score $Y \in {0, 1}$.

In [ ]:
def simple_threshold_data(number_a0, number_a1,
                          a0_threshold, a1_threshold,
                          a0_label, a1_label):

    a0s = np.full(number_a0, a0_label)
    a1s = np.full(number_a1, a1_label)

    a0_scores = np.linspace(0, 1, number_a0)
    a1_scores = np.linspace(0, 1, number_a1)
    score_feature = np.concatenate((a0_scores, a1_scores), axis=None)

    A = np.concatenate((a0s, a1s), axis=None)

    Y_a0 = [x > a0_threshold for x in a0_scores]
    Y_a1 = [x > a1_threshold for x in a1_scores]

    Y = np.concatenate((Y_a0, Y_a1), axis=None)

    X = pd.DataFrame({"credit_score_feature": score_feature,
                      "example_sensitive_feature": A})
    return X, Y, A

We now use the above function to generate our dataset. We have 31 individuals with label $a_0$, and they only require a credit score of 0.2 to get the loan. In contrast, the 21 members of the $a_1$ population require a score of 0.7. The label values for $a_0$ and $a_1$ have to be numeric, but the actual values are not especially important. We set them to 2 and 3, to reduce fiddling with `sklearn`.

In [ ]:
num_samples_a0 = 31
num_samples_a1 = 21

a0_threshold= 0.2
a1_threshold = 0.7

a0_label = 2
a1_label = 3

X, Y, A = simple_threshold_data(num_samples_a0, num_samples_a1, a0_threshold, a1_threshold, a0_label, a1_label)

The following helper function plots the score $Y$ for each of the subpopulations in our data.

In [ ]:
import matplotlib.pyplot as plt
def plot_data(Xs, Ys):
    labels = np.unique(Xs["example_sensitive_feature"])
    
    for l in labels:
        label_string = str(l.item())
        mask = Xs["example_sensitive_feature"] == l
        plt.scatter(Xs[mask].credit_score_feature, Ys[mask], label=str("Label="+label_string))
        plt.xlabel("Credit Score")
        plt.ylabel("Got Loan")
        
    plt.legend()
    plt.show()

Plotting the input data, we can clearly see the bias against the $a_1$ population.

In [ ]:
plot_data(X, Y)

For this notebook, we use a simple binary classifier from `sklearn`. We can train this model on our biased data, and look at the weights the model places on each feature. The fact that the `sensitive_features` has non-zero weight (the second entry in the `coef_` array) tells us that we have a biased model (note that measuring fairness in this way is only valid for this simple example notebook - in the real world, fairness is more complicated).

In [ ]:
unfair_model = LogisticRegression(solver='liblinear', fit_intercept=True)
unfair_model.fit(X, Y, sample_weight=np.ones(len(Y)))

unfair_model.coef_

We can also plot out the predictions for this model. We can see that a few points have changed (which is not unexpected) but the bias definitely remains.

In [ ]:
Y_predict_unfair = unfair_model.predict(X)
plot_data(X, Y_predict_unfair)

## Reducing Unfairness with Grid Search

Now, we move on to attempting to reduce the unfairness in our model using the grid search. This tries a series of different models, parameterized by a Lagrange multiplier $\lambda_i$. For each value of $\lambda$, the algorithm reweights and relabels the input data, and trains a fresh model ($\lambda=0$ corresponds to the unaltered case).

The grid search acts like a normal `sklearn` estimator, implementing `fit()` and `predict()` methods. The `fit()` method performs the grid search, and the best model found (according to a specified selection rule) is used in `predict()` calls. However, after `fit()` is called, there are several extra properties on the estimator. 
- for each grid point we have an entry in the following collections: `lambda_vecs_`, `objectives_`, `gammas_`, `predictors_`.
- `best_idx_` indicates the index of the grid point where grid search found the best solution. The `predict()` method uses the corresponding predictor.

We start by telling the algorithm that we want to try 7 different values of $\lambda$ (which are generated for us).

In [ ]:
first_sweep=GridSearch(LogisticRegression(solver='liblinear', fit_intercept=True),
                       constraints=DemographicParity(),
                       grid_size=7)

first_sweep.fit(X, Y, sensitive_features=A)

We can examine the values of $\lambda_i$ chosen for us:

In [ ]:
lambda_vecs = first_sweep.lambda_vecs_
lambda_vecs[0]

This is rather more than a single value $\lambda$, so what's going on? These results are the outputs of the `Moment` type which drives the reduction approach to removing disparity. There are actually four Lagrange multipliers here, indexed by a tuple (sign, grp, group_id). The 'group_id' field corresponds to the labels $a_0$ and $a_1$, while the 'grp' field is the same in all cases (this is because we have specified Demographic Parity as our disparity criterion). Finally the 'sign' comes from the reductions approach specifying separate multipliers for violations of the disparity criterion from above and below. Both of these are constrained to be positive.

So we have four multipliers - $\lambda_{(+,2)}$, $\lambda_{(-,2)}$, $\lambda_{(+,3)}$ and $\lambda_{(-,3)}$. Without losing generality, we can decide to modify one of these, but not the other, and the `DemographicParity` object we passed to the `GridSearch` constructor chose to make $\lambda_{(+,3)}=\lambda_{(-,3)}=0$. Finally, we can combine the 'above' and 'below' multipliers for the other group and obtain $\lambda_i = \lambda_{(+,2)} - \lambda_{(-,2)}$:

In [ ]:
actual_multipliers = [lambda_vecs[col][("+", "all", 2)]-lambda_vecs[col][("-", "all", 2)] for col in lambda_vecs]
actual_multipliers

And we can look at how the weight the models place on the protected attribute (recall that in the fair case, this would be zero) varies with $\lambda_i$

In [ ]:
first_sweep_sensitive_feature_weights = [
            predictor.coef_[0][1] for predictor in first_sweep.predictors_]
plt.scatter(actual_multipliers, first_sweep_sensitive_feature_weights)
plt.xlabel("Lagrange Multiplier")
plt.ylabel("Weight of Protected Attribute in Model")
plt.show()

We can take a look at the $\lambda$ of the best model found in this sweep, along with the coefficients applied to the features. The weight on the protected attribute is now smaller:

In [ ]:
lambda_best = first_sweep.lambda_vecs_[first_sweep.best_idx_][("+", "all", 2)] - first_sweep.lambda_vecs_[first_sweep.best_idx_][("-", "all", 2)]
print("lambda_best =", lambda_best)
print("coefficients =", first_sweep.predictors_[first_sweep.best_idx_].coef_)

We can also generate predictions from this model. When we plot them, we see that we're much closer to having a fair model, with the two thresholds closer.

In [ ]:
Y_first_predict = first_sweep.predict(X)
plot_data(X, Y_first_predict)

We can do better. For this simple case, we can search $\lambda$ values around that of the best model selected in the first sweep:

In [ ]:
second_sweep_multipliers = np.linspace(lambda_best-0.5, lambda_best+0.5, 31)

iterables = [['+', '-'], ['all'], [a0_label, a1_label]]
midx = pd.MultiIndex.from_product(iterables, names=['sign', 'event', 'group_id'])

second_sweep_lambdas = []
for l in second_sweep_multipliers:
    nxt = pd.Series(np.zeros(4), index=midx)
    if l < 0:
        nxt[("-", "all", 2)] = abs(l)
    else:
        nxt[("+", "all", 2)] = l
    second_sweep_lambdas.append(nxt)
    
multiplier_df = pd.concat(second_sweep_lambdas,axis=1)

And we can use these multipliers in another sweep:

In [ ]:
second_sweep=GridSearch(LogisticRegression(solver='liblinear', fit_intercept=True),
                        constraints=DemographicParity(),
                        grid=multiplier_df)

second_sweep.fit(X, Y, sensitive_features=A)

Once more we can plot the weight placed on the protected attribute as a function of $\lambda$:

In [ ]:
second_sweep_sensitive_feature_weights = [
            predictor.coef_[0][1] for predictor in second_sweep.predictors_]
plt.scatter(second_sweep_multipliers, second_sweep_sensitive_feature_weights)
plt.xlabel("Lagrange Multiplier")
plt.ylabel("Weight of Protected Attribute in Model")
plt.show()


We can look at the $\lambda$ and weights placed on each feature in the best model found from this sweep:

In [ ]:
lambda_best_second = second_sweep.lambda_vecs_[second_sweep.best_idx_][("+", "all", 2)] \
                     -second_sweep.lambda_vecs_[second_sweep.best_idx_][("-", "all", 2)]
print("lambda_best =", lambda_best_second)
print("coefficients =", second_sweep.predictors_[second_sweep.best_idx_].coef_)

And finally, we can obtain a fresh set of predictions from this model. We can see that the threshold is converging on the value 0.2, which was originally specified for just the $a_0$ population:

In [ ]:
Y_second_predict = second_sweep.predict(X)
plot_data(X, Y_second_predict)

In [ ]:
import scrapbook as sb

In [ ]:
sb.glue("best_lambda_second_grid", lambda_best_second)

In [ ]:
sb.glue("best_coeff_second0", second_sweep.predictors_[second_sweep.best_idx_].coef_[0][0])